<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#SQLIte-introduction" data-toc-modified-id="SQLIte-introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>SQLIte introduction</a></span><ul class="toc-item"><li><span><a href="#Connecting/Creating--a-database-sqlite4.connect" data-toc-modified-id="Connecting/Creating--a-database-sqlite4.connect-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Connecting/Creating  a database <code>sqlite4.connect</code></a></span></li><li><span><a href="#Cursor-to-a-database:-conn.cursor" data-toc-modified-id="Cursor-to-a-database:-conn.cursor-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Cursor to a database: <code>conn.cursor</code></a></span></li><li><span><a href="#Executing-SQL-with:-cursor.execute" data-toc-modified-id="Executing-SQL-with:-cursor.execute-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Executing SQL with: <code>cursor.execute</code></a></span></li><li><span><a href="#Fetching-items-from-a-DB" data-toc-modified-id="Fetching-items-from-a-DB-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Fetching items from a DB</a></span></li></ul></li><li><span><a href="#Custom-data-types-in-sqlite" data-toc-modified-id="Custom-data-types-in-sqlite-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Custom data types in sqlite</a></span></li><li><span><a href="#SQLite-Benchmarking" data-toc-modified-id="SQLite-Benchmarking-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>SQLite Benchmarking</a></span></li></ul></div>

In [1]:
import sqlite3

In [2]:
!rm vectors.db

rm: vectors.db: No such file or directory


## SQLIte introduction

### Connecting/Creating  a database `sqlite4.connect`

We can connect to an in-memory database using `.connect(':memory:')`

In [3]:
help(sqlite3.connect)

Help on built-in function connect in module _sqlite3:

connect(...)
    connect(database[, timeout, detect_types, isolation_level,
            check_same_thread, factory, cached_statements, uri])
    
    Opens a connection to the SQLite database file *database*. You can use
    ":memory:" to open a database connection to a database that resides in
    RAM instead of on disk.



In [4]:
conn_memory = sqlite3.connect(':memory:')

In [5]:
conn_memory

We can connect to an in-disk database using `.connect('name_database.db')`.

This creates a file `name_database.db`.

In [6]:
conn = sqlite3.connect('vectors.db')

In [7]:
ls

SQL.ipynb   vectors.db


Note that the file is empty 

In [8]:
!du vectors.db

0	vectors.db


### Cursor to a database: `conn.cursor`

In [9]:
cursor = conn.cursor()

In [10]:

cursor.execute("""CREATE TABLE vectors (
                  id text,
                  price real
                  )""")
conn.commit()
conn.close()

We can see that the file has already the table

In [11]:
!du vectors.db

16	vectors.db


In [12]:
!cat vectors.db

   � �                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                               

### Executing SQL with: `cursor.execute`

In [13]:
import uuid
#a = uuid.uuid4()
#hex_uuid = a.hex
hex_uuid = '12345'
price = 500

In [14]:
conn = sqlite3.connect('vectors.db')
cursor = conn.cursor()

We can insert data into the DB using an `INSERT` clause in SQL

In [15]:
cursor.execute(f"INSERT INTO vectors VALUES ('{hex_uuid}', {price})")
cursor.execute(f"INSERT INTO vectors VALUES ('{45677}', {price})")
cursor.execute(f"INSERT INTO vectors VALUES ('{89999}', {price})")
cursor.execute(f"INSERT INTO vectors VALUES ('{99999}', 300)")

In [16]:
cursor.execute("SELECT * FROM vectors")
cursor.fetchall()

[('12345', 500.0), ('45677', 500.0), ('89999', 500.0), ('99999', 300.0)]

Elements can also be introduced using `execute(sql_query, inputs)`.


This can be done passing:

- SQL query with `?` placeholders and a tuple.
- SQL query with `:name` placeholders and a dictionary

In [17]:
cursor.execute(f"INSERT INTO vectors VALUES (?, ?)", (221,350))

In [18]:
cursor.execute(f"INSERT INTO vectors VALUES (:id, :dollars)", 
               {'id':234,'dollars':325})

In [19]:
conn.commit()
conn.close()

### Fetching items from a DB


- `cursor.fetone()`: retrieve the first item from the query
- `cursor.fetchmany(k)`: retrieve top k items form the query
- `cursor.fetchall()`: retrieve all items from the query



Let's see if the data was stored. 

In [20]:
conn = sqlite3.connect('vectors.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM vectors WHERE price=500")
cursor.fetchone()

('12345', 500.0)

In [21]:
cursor.execute("SELECT * FROM vectors WHERE price=500")
cursor.fetchmany(2)

[('12345', 500.0), ('45677', 500.0)]

In [22]:
conn = sqlite3.connect('vectors.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM vectors WHERE price=500")
cursor.fetchall()

[('12345', 500.0), ('45677', 500.0), ('89999', 500.0)]

We can also call .execute with a pair of inputs as follows:

In [23]:
conn = sqlite3.connect('vectors.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM vectors WHERE price=?", (325,))
cursor.fetchall()

[('234', 325.0)]

In [24]:
conn = sqlite3.connect('vectors.db')
cursor = conn.cursor()
cursor.execute("SELECT * FROM vectors WHERE price=:price", {'price':325})
cursor.fetchall()

[('234', 325.0)]

## Custom data types in sqlite

In [130]:
import numpy as np

class Vector:
    def __init__(self, arg):
        self.arg = arg

    def __str__(self):
        return 'Vector({!r})'.format(self.arg)
    
def adapter(obj: np.ndarray):
    return obj.tobytes()

def converter(data: bytes):
    return np.frombuffer(data)

sqlite3.register_adapter(Vector, adapter)
sqlite3.register_converter(f"{Vector.__name__}", converter)

In [132]:

con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("CREATE TABLE test(v Vector)")
cur.execute("INSERT into test(v) values (?)", (np.array([1.,2.,3.,4.]),))
cur.execute("INSERT into test(v) values (?)", (np.array([2.,3.,4.,5.]),))
cur.execute('SELECT v from test')
cur.fetchall()

[(array([1., 2., 3., 4.]),), (array([2., 3., 4., 5.]),)]

This can cause problems if the datatype is not correctly set

In [172]:
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("CREATE TABLE test(v Vector)")
cur.execute("INSERT into test(v) values (?)", (np.array([1,2,3,4]),))
cur.execute("INSERT into test(v) values (?)", (np.array([2,3,4,5]),))
cur.execute('SELECT v from test')
cur.fetchall()

[(array([4.9e-324, 9.9e-324, 1.5e-323, 2.0e-323]),),
 (array([9.9e-324, 1.5e-323, 2.0e-323, 2.5e-323]),)]

In [173]:
cur.execute('select v from test')
vectors = []
for v, in cur.fetchall():
    print(v.shape, v)
    assert isinstance(v, np.ndarray)
    vectors.append(v)

(4,) [4.9e-324 9.9e-324 1.5e-323 2.0e-323]
(4,) [9.9e-324 1.5e-323 2.0e-323 2.5e-323]


To get the correct vector data needs to be interpreted in the correct data type

In [174]:
print(np.frombuffer(vectors[0],dtype=np.int64))
print(np.frombuffer(vectors[1],dtype=np.int64))

[1 2 3 4]
[2 3 4 5]


In [154]:
def adapt_array(arr):
    return arr.tobytes()

def convert_array(bytes):
    return np.frombuffer(bytes)

sqlite3.register_adapter(np.array, adapt_array)    
sqlite3.register_converter("array", convert_array)

In [164]:
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("CREATE TABLE test(v BLOB)")
cur.execute("INSERT into test(v) values (?)", (np.array([1,2,3,4]),))
cur.execute("INSERT into test(v) values (?)", (np.array([2,3,4,5]),))
cur.execute('SELECT v from test')
cur.fetchall()

cur.execute('select v from test')
vectors = []
for v, in cur.fetchall():
    vectors.append(v)

In [167]:
print(np.frombuffer(vectors[0],dtype=np.int64))
print(np.frombuffer(vectors[1],dtype=np.int64))

[1 2 3 4]
[2 3 4 5]


In [170]:
%%timeit
cur.execute('SELECT v from test')
cur.fetchall()

3.05 µs ± 90.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [196]:
con = sqlite3.connect(":memory:", detect_types=sqlite3.PARSE_DECLTYPES)
cur = con.cursor()
cur.execute("CREATE TABLE vecblob(v BLOB)")
cur.execute("INSERT into vecblob(v) values (?)", (np.array([1,2,3,4]),))
cur.execute("INSERT into vecblob(v) values (?)", (np.array([2,3,4,5]),))
cur.execute('SELECT v from vecblob')
cur.fetchall()

[(b'\x01\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00',),
 (b'\x02\x00\x00\x00\x00\x00\x00\x00\x03\x00\x00\x00\x00\x00\x00\x00\x04\x00\x00\x00\x00\x00\x00\x00\x05\x00\x00\x00\x00\x00\x00\x00',)]

## SQLite Benchmarking

In [25]:
conn_mem = sqlite3.connect(':memory:')
cursor_mem = conn_mem.cursor()

cursor_mem.execute("""CREATE TABLE vectors (
                  id text,
                  price real
                  )""")

cursor_mem.execute(f"INSERT INTO vectors VALUES ('{hex_uuid}', {price})")
cursor_mem.execute(f"INSERT INTO vectors VALUES ('{45677}', {price})")
cursor_mem.execute(f"INSERT INTO vectors VALUES ('{89999}', {price})")
cursor_mem.execute(f"INSERT INTO vectors VALUES ('{99999}', 300)")

#cursor_mem.commit()
#cursor_mem.close()

We can see that accessing a DB that is in memory is faster than one that is stored on disk

In [26]:
%%timeit
cursor_mem.execute("SELECT * FROM vectors WHERE price=500")
cursor_mem.fetchall()

4.85 µs ± 346 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [27]:
%%timeit
cursor.execute("SELECT * FROM vectors WHERE price=500")
cursor.fetchall()

21.9 µs ± 1.79 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [28]:
cursor_mem.execute("SELECT * FROM vectors WHERE price=500")
cursor_mem.fetchall()

[('12345', 500.0), ('45677', 500.0), ('89999', 500.0)]